In [1]:
from TM1py.Services import TM1Service
from TM1py.Utils import Utils

tm1_src = TM1Service(address="localhost", port=30012, ssl=False, user="admin", password="")
tm1_tgr = TM1Service(address="localhost", port=30012, ssl=False, user="admin", password="")
cubename_src='Sales'
cubename_tgr='Sales less product and business unit'

mdx='''
     SELECT 
       {[Month].[Month].members}
   
      ON COLUMNS , 
      {TM1FilterByLevel({TM1SubSetAll([Location])}, 0)}
   
      PROPERTIES [Location].[Location].[Full Description]  ON ROWS 
      FROM [Sales] 
      WHERE 
          (
           [Year].[Year].[2012/2013],
           [Measure_Sales].[Measure_Sales].[Volume],
           [Data Source].[Data Source].[Input],
           [Business Unit].[Business Unit].[81],
           [Version].[Version].[Actual]
          )

         '''
#元组字典
#cells=tm1_src.cubes.cells.execute_mdx(mdx,skip_cell_properties=True,skip_rule_derived_cells=True,skip_consolidated_cells=True,skip_zeros=True,element_unique_names=False)
cells=tm1_src.cubes.cells.execute_mdx(mdx,skip_rule_derived_cells=True,skip_zeros=True)
df = Utils.build_pandas_dataframe_from_cellset(cells)
#维度收拢
df_less_dim=df.reset_index().drop(['Business Unit','Product'],axis=1)

#Clear data
mdx_tgr='''
         SELECT 
           {[Month].[Month].Members} 
          ON COLUMNS , 
           {[Location].[Location].Members} 
           PROPERTIES [Location].[Location].[Full Description]  ON ROWS 
          FROM [Sales less product and business unit] 
            WHERE 
              (
               [Year].[Year].[2012/2013],
               [Version].[Version].[Actual],
               [Data Source].[Data Source].[Input],
               [Measure_Sales].[Measure_Sales].[Volume]
              )   
            
            
            '''
tm1_tgr.cells.clear_with_mdx(cubename_tgr,mdx_tgr)
#Writing to tgr Cube
tm1_tgr.cells.write_dataframe(
        cube_name=cubename_tgr,
        #restore columns
        data=df_less_dim,
        dimensions=['Year','Month','Version','Location','Data Source','Measure_Sales'],
        increment=False,
        deactivate_transaction_log=True,
        reactivate_transaction_log=True,
        sandbox_name=None,
        use_ti=True,
        skip_non_updateable=True,
        use_changeset=False)


In [49]:
df

Values
Year      Month      Version Location Business Unit Product      Data Source Measure_Sales        
2012/2013 All Months Actual  102      81            All Products Input       Volume           4680
          Jul        Actual  102      81            All Products Input       Volume             60
          Aug        Actual  102      81            All Products Input       Volume            120
          Sep        Actual  102      81            All Products Input       Volume            180
          Oct        Actual  102      81            All Products Input       Volume            240
...                                                                                            ...
          Jan YTG    Actual  111      81            All Products Input       Volume           3000
          Feb YTG    Actual  111      81            All Products Input       Volume           2520
          Mar YTG    Actual  111      81            All Products Input       Volume           1980
          Apr YTG    Actual  111      81            All Products Input       Volume           1380
          May YTG    Actual  111      81            All Products Input       Volume            720

[430 rows x 1 columns]

In [52]:
df.reset_index()

,Year,Month,Version,Location,Business Unit,Product,Data Source,Measure_Sales,Values
0,2012/2013,All Months,Actual,102,81,All Products,Input,Volume,4680
1,2012/2013,Jul,Actual,102,81,All Products,Input,Volume,60
2,2012/2013,Aug,Actual,102,81,All Products,Input,Volume,120
3,2012/2013,Sep,Actual,102,81,All Products,Input,Volume,180
4,2012/2013,Oct,Actual,102,81,All Products,Input,Volume,240
...,...,...,...,...,...,...,...,...,...
425,2012/2013,Jan YTG,Actual,111,81,All Products,Input,Volume,3000
426,2012/2013,Feb YTG,Actual,111,81,All Products,Input,Volume,2520
427,2012/2013,Mar YTG,Actual,111,81,All Products,Input,Volume,1980
428,2012/2013,Apr YTG,Actual,111,81,All Products,Input,Volume,1380


In [48]:
df_less_dim

,Year,Month,Version,Location,Data Source,Measure_Sales,Values
0,2012/2013,All Months,Actual,102,Input,Volume,4680
1,2012/2013,Jul,Actual,102,Input,Volume,60
2,2012/2013,Aug,Actual,102,Input,Volume,120
3,2012/2013,Sep,Actual,102,Input,Volume,180
4,2012/2013,Oct,Actual,102,Input,Volume,240
...,...,...,...,...,...,...,...
425,2012/2013,Jan YTG,Actual,111,Input,Volume,3000
426,2012/2013,Feb YTG,Actual,111,Input,Volume,2520
427,2012/2013,Mar YTG,Actual,111,Input,Volume,1980
428,2012/2013,Apr YTG,Actual,111,Input,Volume,1380


#### 如需调整dataframe列顺序与Cube维度顺序一致,参考代码

In [ ]:
col_name=['Senario','Version','Year','Month','Settle Account Year','Settle Account Month','Day',
          '货币','平台行为','sku','Country','PlatForm Number','Measure','Value']

df_final=df_clean_melt.reindex(columns=col_name)

In [46]:
tm1_src = TM1Service(address="localhost", port=30012, ssl=False, user="admin", password="")
leaves = tm1_src.elements.get_leaf_element_names("Month", "Month")

In [47]:
leaves

['Jul',
 'Aug',
 'Sep',
 'Quarter 1_n',
 'Oct',
 'Nov',
 'Dec',
 'Jan',
 'Feb',
 'Mar',
 'Quarter 3_n',
 'Apr',
 'May',
 'Jun',
 'Quarter 4_n',
 'Unspecified Month',
 'Quarter 2_n']